In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_var = 20      # number of variables
num_ineq = 20     # number of constraints
num_data = 10000  # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# data sample from uniform distribution
b_samples = torch.from_numpy(np.random.uniform(-1, 1, size=(num_data, num_ineq))).float()
data = {"b":b_samples}
# data split
from src.utlis import data_split
data_train, data_test, data_dev = data_split(data, test_size=test_size, val_size=val_size)

In [5]:
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [6]:
from src.problem import msAckley
model = msAckley(num_var, num_ineq, timelimit=60)

## Learnable Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 50   # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 64            # width of hidden layers for solution mapping
lr = 1e-5             # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmAckley
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmAckley(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs=epochs, patience=patience, warmup=warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 300.43
Epoch 1, Validation Loss: 277.00
Epoch 2, Validation Loss: 272.23
Epoch 3, Validation Loss: 271.55
Epoch 4, Validation Loss: 253.33
Epoch 5, Validation Loss: 246.14
Epoch 6, Validation Loss: 243.06
Epoch 7, Validation Loss: 238.43
Epoch 8, Validation Loss: 210.72
Epoch 9, Validation Loss: 189.53
Epoch 10, Validation Loss: 188.85
Epoch 11, Validation Loss: 182.82
Epoch 12, Validation Loss: 169.85
Epoch 13, Validation Loss: 137.30
Epoch 14, Validation Loss: 121.45
Epoch 15, Validation Loss: 111.76
Epoch 16, Validation Loss: 83.41
Epoch 17, Validation Loss: 65.32
Epoch 18, Validation Loss: 52.16
Epoch 19, Validation Loss: 39.00
Epoch 20, Validation Loss: 29.73
Epoch 21, Validation Loss: 20.61
Epoch 22, Validation Loss: 16.54
Epoch 23, Validation Loss: 14.36
Epoch 24, Validation Loss: 13.07
Epoch 25, Validation Loss: 12.51
Epoch 26, Validation Loss: 12.24
Epoch 27, Validation Loss: 11.96
Epoch 28, Validation Loss: 11.93
Epoch 29, Validation Loss: 12.03
Epoc

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ak_lr_20-20.csv")

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:05<00:00, 167.65it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean     10.212751          0.003737      0.002686
std       0.708695          0.024318      0.000853
min       8.284150          0.000000      0.001000
25%       9.618884          0.000000      0.002002
50%      10.235100          0.000000      0.002515
75%      10.722263          0.000000      0.003003
max      12.271281          0.424752      0.010526
Number of infeasible solution: 39


## Learnable Threshold

In [12]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [13]:
# hyperparameters
penalty_weight = 50   # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 64            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [14]:
# set problem
import neuromancer as nm
from src.problem import nmAckley
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmAckley(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [15]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs=epochs, patience=patience, warmup=warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 246.31
Epoch 1, Validation Loss: nan
Epoch 2, Validation Loss: nan
Epoch 3, Validation Loss: nan
Epoch 4, Validation Loss: nan
Epoch 5, Validation Loss: nan
Epoch 6, Validation Loss: nan
Epoch 7, Validation Loss: nan
Epoch 8, Validation Loss: nan
Epoch 9, Validation Loss: nan
Epoch 10, Validation Loss: nan
Epoch 11, Validation Loss: nan
Epoch 12, Validation Loss: nan
Epoch 13, Validation Loss: nan
Epoch 14, Validation Loss: nan
Epoch 15, Validation Loss: nan
Epoch 16, Validation Loss: nan
Epoch 17, Validation Loss: nan
Epoch 18, Validation Loss: nan
Epoch 19, Validation Loss: nan
Epoch 20, Validation Loss: nan
Epoch 21, Validation Loss: nan
Epoch 22, Validation Loss: nan
Epoch 23, Validation Loss: nan
Epoch 24, Validation Loss: nan
Epoch 25, Validation Loss: nan
Epoch 26, Validation Loss: nan
Epoch 27, Validation Loss: nan
Epoch 28, Validation Loss: nan
Epoch 29, Validation Loss: nan
Epoch 30, Validation Loss: nan
Epoch 31, Validation Loss: nan
Epoch 32, Valid

In [16]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ak_lt_20-20.csv")

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:06<00:00, 162.45it/s]


       Obj Val  Constraints Viol  Elapsed Time
count      0.0            1000.0   1000.000000
mean       NaN               0.0      0.002537
std        NaN               0.0      0.000733
min        NaN               0.0      0.000996
25%        NaN               0.0      0.002000
50%        NaN               0.0      0.002506
75%        NaN               0.0      0.003000
max        NaN               0.0      0.007030
Number of infeasible solution: 0
